In [1]:
import numpy as np
import scipy as sp
import scipy.sparse as spsp
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# Diffusion function
g = lambda s: 1/(1+s)
g_exp = lambda s: np.exp(-s**2)

f = lambda x, y: x + y

def generate_squares2D(N, M):
    I = np.zeros((N+2, M+2))
    I[:N//2+1, :M//2+1] = 80
    I[:N//2+1, -(M//2+1):] = 190
    I[-(N//2+1):, :M//2+1] = 140
    I[-(N//2+1):, -(M//2+1):] = 230
    return I

def add_noise2D(I, scale = 10):
    N, M = I.shape
    N -= 2
    M -= 2
    I[1:-1, 1:-1] += np.random.rand(N,M) * 5
    return I
    

# Create a random test image
def generate_random2D(N, M, scale = 10):
    # Generate image of 4 squares
    I = generate_squares2D(N, M)
    
    # Add noise to picture
#     I[1:-1, 1:-1] += np.random.rand(N,M) * 5
    add_noise2D(I)
    return I

def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

def image_display(V, n, m, cmap = "gray"):
    image = V.reshape((n, m))
    plt.imshow(image, cmap)
    plt.show()

In [8]:
Dy

array([[-31.5,  -0. ,  -0. , ...,   0. ,   0. ,   0. ],
       [ -0. , -31.5,  -0. , ...,   0. ,   0. ,   0. ],
       [ -0. ,  -0. , -31.5, ...,   0. ,   0. ,   0. ],
       ..., 
       [  0. ,   0. ,   0. , ...,  31.5,   0. ,   0. ],
       [  0. ,   0. ,   0. , ...,   0. ,  31.5,   0. ],
       [  0. ,   0. ,   0. , ...,   0. ,   0. ,  31.5]])

In [6]:
# Spatial discretization
M = 20
N = 20
K = (M+2) * (N+2)
dx = 1/(M+1)
dy = 1/(N+1)

# Initial conditions
I = generate_random2D(N, M, scale = 2)
# I = load_image("lena-64x64.jpg") + np.random.randint(0, 20, (M+2, N+2))


# Number of iterations, and timestep
T = 10
# dt = 1e-1
dt = 1e-7
r = dt/(2*(dx**2 + dy**2))

# Block matrix x-derivative
Bx = -1 * np.eye(M+2, k = -1) + np.eye(M+2, k = 1)
Bx[0, :3] = [-3, 4, -1]
Bx[-1, -3:] = [1, -4, 3]

# Differentiation matrix x-direction
Dx = np.zeros((K, K))
for i in range(N+2):
    Dx[i*(M+2):(i+1)*(M+2), i*(M+2):(i+1)*(M+2)] = Bx  
Dx /= 2*dx

# Differentiation matrix y-direction
Dy = -np.eye(K, k = -(M+2)) + np.eye(K, k = M+2)
Dy[:(M+2), :3*(M+2)] = np.hstack((-3*np.identity(M+2), 4*np.identity(M+2), -np.identity(M+2)))
Dy[-(M+2):, -3*(M+2):] = np.hstack((np.identity(M+2), -4*np.identity(M+2), 3*np.identity(M+2)))
Dy /= 2*dy

Ξx = np.zeros((K, K))
Ωx = np.zeros((K, K))
Γx = np.zeros((K, K))

Ξy = np.zeros((K, K))
Ωy = np.zeros((K, K))
Γy = np.zeros((K, K))


# Fill in matrices
for i in range(1, N+1):
    Ξx[i*(M+2):(i+1)*(M+2), i*(M+2):(i+1)*(M+2)] = Xx

for i in range(1, N+1):
    Ωx[i*(M+2):(i+1)*(M+2), i*(M+2):(i+1)*(M+2)] = Mx

for i in range(1, N+1):
    Γx[i*(M+2):(i+1)*(M+2), i*(M+2):(i+1)*(M+2)] = Fx

for i in range(1, N+1):
    Ξy[i*(M+2)+1:(i+1)*(M+2)-1, (i-1)*(M+2)+1:i*(M+2)-1] = np.identity(M)
    Ξy[i*(M+2)+1:(i+1)*(M+2)-1, i*(M+2)+1:(i+1)*(M+2)-1] = np.identity(M)

for i in range(1, N+1):
    Ωy[i*(M+2)+1:(i+1)*(M+2)-1, (i-1)*(M+2)+1:i*(M+2)-1] = -np.identity(M)
    Ωy[i*(M+2)+1:(i+1)*(M+2)-1, i*(M+2)+1:(i+1)*(M+2)-1] = -2*np.identity(M)
    Ωy[i*(M+2)+1:(i+1)*(M+2)-1, (i+1)*(M+2)+1:(i+2)*(M+2)-1] = -np.identity(M)

for i in range(1, N+1):
    Γy[i*(M+2)+1:(i+1)*(M+2)-1, i*(M+2)+1:(i+1)*(M+2)-1] = np.identity(M)
    Γy[i*(M+2)+1:(i+1)*(M+2)-1, (i+1)*(M+2)+1:(i+2)*(M+2)-1] = np.identity(M)
    
# Initiate grid
U = np.zeros((T, K))
U[0] = I.reshape(K)

In [7]:
Dx

array([[-31.5,  42. , -10.5, ...,   0. ,   0. ,   0. ],
       [-10.5,   0. ,  10.5, ...,   0. ,   0. ,   0. ],
       [  0. , -10.5,   0. , ...,   0. ,   0. ,   0. ],
       ..., 
       [  0. ,   0. ,   0. , ...,   0. ,  10.5,   0. ],
       [  0. ,   0. ,   0. , ..., -10.5,   0. ,  10.5],
       [  0. ,   0. ,   0. , ...,  10.5, -42. ,  31.5]])

## Implisitt

In [ ]:
# Perform iteration
dt = 5e-5
r = dt/(2*(dx**2 + dy**2))
for it in range(T-1):
    G = g(Dx.dot(U[it])**2 + Dy.dot(U[it])**2)
        
    ξx = Ξx.dot(G)
    ωx = Ωx.dot(G)
    γx = Γx.dot(G)

    ξy = Ξy.dot(G)
    ωy = Ωy.dot(G)
    γy = Γy.dot(G)
    
    # Construct scheme matrix
    x_diags = (ξx[1:], ωx, γx[:-1])
    y_diags = (ξy[(M+2):], ωy, γy[:-(M+2)])
    Ax = spsp.diags(x_diags, (-1, 0, 1)).toarray()/(2*dx**2)
    Ay = spsp.diags(y_diags, (-(M+2), 0, M+2)).toarray()/(2*dy**2)
    A = Ax + Ay
    
    U[it+1] = np.linalg.solve((np.identity(K) + r * A), U[it])
    
    # Debug
    if it%1 == 0:
        print(it)
        plt.figure(figsize=(16, 8))
        plt.subplot(143)
        plt.imshow(G.reshape(N+2, M+2), cmap = "gray")
        plt.subplot(144)
        plt.imshow(A.dot(U[it]).reshape(N+2, M+2), cmap = "gray")
        plt.subplot(142)
        plt.imshow((Dy.dot(U[0])**2 + Dx.dot(U[0])**2).reshape(N+2, M+2), cmap = "gray")
        plt.subplot(141)
        plt.imshow(U[it].reshape(N+2, M+2), cmap = "gray")
        plt.show()

In [ ]:
np.linalg.norm(U[6]-U[0])

In [ ]:
# Display initial conditinos
# plt.imshow(U[0].reshape(N+2, M+2), cmap = "gray")
# plt.show()

# Perform iterations
for it in range(T-1):
    G = g(Dx.dot(U[it])**2 + Dy.dot(U[it])**2)
        
    ξx = Ξx.dot(G)
    ωx = Ωx.dot(G)
    γx = Γx.dot(G)

    ξy = Ξy.dot(G)
    ωy = Ωy.dot(G)
    γy = Γy.dot(G)
    
    # Construct scheme matrix
    x_diags = (ξx[1:], ωx, γx[:-1])
    y_diags = (ξy[(M+2):], ωy, γy[:-(M+2)])
    Ax = spsp.diags(x_diags, (-1, 0, 1)).toarray()/(2*dx**2)
    Ay = spsp.diags(y_diags, (-(M+2), 0, M+2)).toarray()/(2*dy**2)
    A = Ax + Ay
    
    U[it+1] = (np.identity(K) - r * A).dot(U[it])
    
    # Debug
    if it%1 == 0:
        print(it)
        plt.figure(figsize=(12, 8))
        plt.subplot(141)
        plt.imshow(G.reshape(N+2, M+2), cmap = "gray")
        plt.subplot(142)
        plt.imshow(A.dot(np.ones(K)).reshape(N+2, M+2), cmap = "gray")#A.dot(U[it]).reshape(N+2, M+2), cmap = "gray")
        plt.subplot(143)
        plt.imshow((Dy.dot(U[0])**2 + Dx.dot(U[0])**2).reshape(N+2, M+2), cmap = "gray")
        plt.subplot(144)
        plt.imshow(U[it].reshape(N+2, M+2), cmap = "gray")
        plt.show()